In [1]:
import pandas as pd
import numpy as np

In [2]:
# Use pandas to load real_estate_dataset.csv
df = pd.read_csv('real_estate_dataset.csv')

# get the number of samples and features
n_samples, n_features = df.shape

print(f"Number of columns: {n_features}")

# get the names of the coloumns
columns = df.columns

# save the column names to a file fir accessing later as a text file
np.savetxt('columns.txt', columns, fmt='%s')

# use only square_feet, garage_size, location_score, distance_to_center as features:
X = df[['Square_Feet', 'Garage_Size', 'Location_Score', 'Distance_to_Center']]

# use price as the target
y = df['Price'].values

print(f"Shape of X: {X.shape}\n")
print(f"data type of X: {X.dtypes}")

#get the number of samples and features in X
n_samples, n_features = X.shape

Number of columns: 12
Shape of X: (500, 4)

data type of X: Square_Feet           float64
Garage_Size             int64
Location_Score        float64
Distance_to_Center    float64
dtype: object


In [3]:
# Build a linear model to predict price from the four features in X
# make an array of coefs of the size of (n_features + 1), Initialise to 1.

coefs = np.ones(n_features + 1)

# predict the price for each sample in X
predictions_bydefn = X @ coefs[1:] + coefs[0]

# append a columns of ones to X
X = np.hstack([np.ones((n_samples, 1)), X])

# predict the price for each sample in X
predictions = X @ coefs

In [6]:
# see if all entries in predictions_bydefn and predictions are the same
is_same = np.allclose(predictions_bydefn, predictions)
print(f"Are the predictions same? {is_same}")

Are the predictions same? True


In [7]:
# calculate the error using predictions and y
errors = y - predictions

# calculate the relative error
relative_errors = errors / y

# calculate the mean of squares of error using a loop
loss_loop = 0
for i in range(n_samples):
    loss_loop += errors[i] ** 2

loss_loop = loss_loop/n_samples

# calculate the mean of squares of error using matrix operations
loss_matrix = np.transpose(errors) @ errors / n_samples

# check if the two methods give the same result
is_diff = np.allclose(loss_loop, loss_matrix)
print(f"Are the two methods the same? {is_diff}")

# print the size of errors, and its L2 norm
print(f"Size of errors: {errors.shape}")
print(f"L2 norm of errors: {np.linalg.norm(errors)}")
print(f"L2 norm of relative errors: {np.linalg.norm(relative_errors)}")

Are the two methods the same? True
Size of errors: (500,)
L2 norm of errors: 13297007.321853263
L2 norm of relative errors: 22.35214323542266


In [ ]:
# What is my optimisation problem? 
# I want to find the coefficients that minimise the mean of squares of errors
# This is called as Least Squares problem

# Aside
# Nu = f (Re, Pr); Nu = \alpha Re^m Pr^n, we want to find \alpha, m, n that best fit the data

# Objective function: f(coefs) = 1/n_samples * \sum_{i=1}^{n_samples} (y_i - \sum_{j=0}^{n_features} coefs[j] * X[i, j])^2

# What is a solution?
# A solution is a set of coefficients that minimises the objective function

# How do I find a solution?
# By searching for the coeeficients at which the gradient of the objective function is zero
# Or I can set the gradient of the objective function to zero and solve for the coefficients

# write the loss_matrix in terms of data and coefs
loss_matrix = (y - X @ coefs).T @ (y - X @ coefs) / n_samples

# calculate the gradient of the loss with respect to coefs
grad_matrix = -2/n_samples * X.T @ (y - X @ coefs)

# we set grad_matrix to zero and solve for coefs
# X.T @ y = X.T @ X @ coefs
# X.T @ X @ coefs = X.T @ y  (This is called the normal equation)
# coefs = (X.T @ X)^-1 @ X.T @ y

coefs = np.linalg.inv(X.T @ X) @ X.T @ y
# save coefs to a file for viewing later
np.savetxt('coefs.txt', coefs)

# calculate the predictions using the new coefs
predictions_model = X @ coefs

# calculate the errors using the new predictions
errors = y - predictions_model

# # calculate the relative errors
# relative_errors = errors / y

# print L2 norm of the errors_model
print(f"L2 norm of errors_model: {np.linalg.norm(errors)}")
# print L2 norm of the relative errors
print(f"L2 norm of relative errors: {np.linalg.norm(relative_errors)}")

# Use all the features in the dataset to build a linear model to predict price
X = df.drop('Price', axis=1).values
y = df['Price'].values

# get the number of samples and features in X
n_samples, n_features = X.shape
print(f"Number of samples, features: {n_samples, n_features}")
# solve the linear model using the normal equation
X = np.hstack([np.ones((n_samples, 1)), X])
coefs = np.linalg.inv(X.T @ X) @ X.T @ y

# save coefs to a file names coefs_all.txt
np.savetxt('coefs_all.txt', coefs, delimiter=',')


### STUDY JACOBIAN AND HESSIAN. STUDY MATRIX CALCULUS

L2 norm of errors_model: 2240271.803752977
L2 norm of relative errors: 22.35214323542266


In [ ]:
# calculate the rank of X.T @ X
rank_XTX = np.linalg.matrix_rank(X.T @ X)
print(f"Rank of X.T @ X: {rank_XTX}")

# solve the normal equation using matrix decomposition
# QR FACTORISATION
Q, R = np.linalg.qr(X)

print(f"Shape of Q: {Q.shape}") # Q is orthogonal (study)
print(f"Shape of R: {R.shape}")

# write R to a file named R.csv
np.savetxt('R.csv', R, delimiter=',')   # R is upper triangular

# R*coefs = b

sol =Q.T @ Q # identity matrix
np.savetxt('sol.csv', sol, delimiter=',')

# X = QR
# X.T @ X = R.T @ Q.T @ Q @ R = R.T @ R
# X.T @ y = R.T @ Q.T @ y
# R.coefs = Q.T @ y

b = Q.T @ y
print(f"Shape of b: {b.shape}")
print(f"Shape of R: {R.shape}")

#coeffs_qr = np.linalg.inv(R) @ b

# loop to solve R*coeffs = b using back substitution
coeffs_qr_loop = np.zeros(n_features + 1)

for i in range(n_features, -1, -1):
    coeffs_qr_loop[i] = b[i]
    for j in range(i+1, n_features + 1):
        coeffs_qr_loop[i] -= R[i, j] * coeffs_qr_loop[j]

    coeffs_qr_loop[i] = coeffs_qr_loop[i] / R[i, i]
# coeffs_qr_loop = coeffs_qr_loop / np.diag(R)

# save coeffs_qr_loop to a file named coeffs_qr_loop.csv
np.savetxt('coeffs_qr_loop.csv', coeffs_qr_loop, delimiter=',')

# Solve the normal equation using SVD
U, S, Vt = np.linalg.svd(X, full_matrices=False)
# X = U S V^T  (Note that these are transposes and inverses)

# Eigen Decomposition of a square matrix
# A = V D V^T
# A^-1 = V D^-1 V^T
# X * coeffs = y
# A = X^T X

# Normal Equation: X^T X * coeffs = X^T y
# Xdagger = (X^T X)^-1 X^T 

# TO COMPLETE: CALCULATE THE COEFFS_SVD USING THE PSEUDO INVERSE OF X

S_inverse = np.diag(1/S)
coeffs_svd = Vt.T @ S_inverse @ U.T @ y

# Calculate errors and predictions using the new coefs
predictions_svd = X @ coeffs_svd
errors_svd = y - predictions_svd

# L2 norm of errors_svd
print(f"L2 norm of errors_svd: {np.linalg.norm(errors_svd)}")

# L2 norm of relative errors_svd
relative_errors_svd = errors_svd / y
print(f"L2 norm of relative errors_svd: {np.linalg.norm(relative_errors_svd)}")

Rank of X.T @ X: 5
Shape of Q: (500, 5)
Shape of R: (5, 5)
Shape of b: (5,)
Shape of R: (5, 5)
L2 norm of errors_svd: 2240271.803752977
L2 norm of relative errors_svd: 4.32709776267726
